In [ ]:
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_metadata.proto.v0 import schema_pb2

In [ ]:
# Path to the saved model
MODEL_DIR = 'serving_model_dir/xss-detection-model'
TRANSFORM_GRAPH_DIR = 'pipelines/xss-pipeline/Transform/transform_graph'
SCHEMA_FILE = 'pipelines/xss-pipeline/SchemaGen/schema.pbtxt'

# Load the saved model
model = tf.keras.models.load_model(MODEL_DIR)

# Load the transform graph
transform = tft.TFTransformOutput(TRANSFORM_GRAPH_DIR)

# Load the schema
schema = schema_pb2.Schema()
with open(SCHEMA_FILE, 'rb') as f:
    schema.ParseFromString(f.read())

In [ ]:
# Function to preprocess input data
def preprocess(input_data):
    feature_spec = tft.TFTransformOutput(TRANSFORM_GRAPH_DIR).raw_feature_spec()
    feature_spec.pop('Label')
    raw_features = tf.io.parse_example(input_data, feature_spec)
    transformed_features = transform.transform_raw_features(raw_features)
    return transformed_features

# Function to make predictions
def predict(input_sentences):
    serialized_inputs = [tf.train.Example(features=tf.train.Features(feature={
        'Sentence': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.compat.as_bytes(sentence)]))
    })).SerializeToString() for sentence in input_sentences]
    transformed_inputs = preprocess(serialized_inputs)
    predictions = model.predict(transformed_inputs)
    return predictions

# Make Prediction Request

In [ ]:
# Example sentences for prediction
example_sentences = [
    '<div draggable=""true"" contenteditable>drag me</div><image ondragover=alert(1) contenteditable>drop here</image>', # XSS
    '</span> <span class=""reference-text""><a href=""#CITEREFLangley2011"">Langley 2011 </a>.' # not a XSS
]

# Make predictions
predictions = predict(example_sentences)
print('Predictions:', predictions)